# Agrupaciones de datos

Las agrupaciones son operaciones necesarias para analizar datos, ya que permiten extraer información en función de datos categóricos de nuestro dataframe.

Cargaremos los datos llamados _experiment.csv_ que podemos encontrar en el siguiente [enlace](https://raw.githubusercontent.com/bmalcover/MADM2019/master/data/experiment.csv)

In [1]:

import pandas as pd
import numpy as np

df= pd.read_csv("data/experiment.csv")
df

,Nombre,Apellidos,Altura,Sexo,Nacimiento,Cof,Categoria
0,Will,Smith,1.43,M,10/10/1920,0.19,laboral
1,Jon,Snow,1.98,M,10/1/1970,0.98,laboral
2,Laia,Ramirez,1.87,F,09/10/1987,0.76,cap6
3,Luzy,Raim,1.67,F,23/07/1979,0.56,cap6
4,Fein,Mang,1.78,M,12/03/1937,0.27,cap6
5,Victor,Colom,1.78,M,22/09/1957,0.97,cap8


En el siguiente ejemplo agrupamos los datos según el sexo de la persona mediante el método `groupby` que devuelve un `DataFrame` agrupado:

In [2]:
bySex = df.groupby('Sexo')
type(bySex)

pandas.core.groupby.generic.DataFrameGroupBy

El atributo `groups` nos muestra los grupos hemos creado:

In [3]:
# Podemos saber los grupos realizados y que índices del dataframe tienen.
bySex.groups # nos proporciona un diccionario


{'F': [2, 3], 'M': [0, 1, 4, 5]}

In [9]:
bySex.describe()

Altura                                                     Cof          \
      count    mean       std   min     25%   50%   75%   max count    mean   
Sexo                                                                          
F       2.0  1.7700  0.141421  1.67  1.7200  1.77  1.82  1.87   2.0  0.6600   
M       4.0  1.7425  0.228674  1.43  1.6925  1.78  1.83  1.98   4.0  0.6025   

                                                
           std   min   25%   50%     75%   max  
Sexo                                            
F     0.141421  0.56  0.61  0.66  0.7100  0.76  
M     0.431383  0.19  0.25  0.62  0.9725  0.98

Esto nos permite realizar operaciones de filtrado con base a los grupos que hemos creado:

In [4]:
dfM = df.loc[bySex.groups['M'].values] #Recorda que "loc" accedeix per index de fila
dfM

,Nombre,Apellidos,Altura,Sexo,Nacimiento,Cof,Categoria
0,Will,Smith,1.43,M,10/10/1920,0.19,laboral
1,Jon,Snow,1.98,M,10/1/1970,0.98,laboral
4,Fein,Mang,1.78,M,12/03/1937,0.27,cap6
5,Victor,Colom,1.78,M,22/09/1957,0.97,cap8


In [7]:
df[df.Sexo=="M"] # es lo  mismo!

,Nombre,Apellidos,Altura,Sexo,Nacimiento,Cof,Categoria
0,Will,Smith,1.43,M,10/10/1920,0.19,laboral
1,Jon,Snow,1.98,M,10/1/1970,0.98,laboral
4,Fein,Mang,1.78,M,12/03/1937,0.27,cap6
5,Victor,Colom,1.78,M,22/09/1957,0.97,cap8


### Funciones de agregación en grupos.

El método 'aggregate' nos permite crear variables de agregación en la tabla obtenida con 'groupby'. Indicaremos la información que deseamos obtener de cada columna utilizando un diccionario. Especificamos la función que aplicaremos a los datos de cada grupo en cada columna para obtener un único valor.

- Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.aggregate.html

Para ejemplificar esta sección, agruparemos el _dataframe_ por `Categoria` laboral. En este caso para la columna `Altura` consultamos la suma de las alturas del grupo y `Cof` la media.

La función `aggregate` nos permite crear variables de agregación sobre la tabla obtenida con `groupby`. Indicaremos la información que queremos obtener de cada columna con un diccionario. Especificamos la función que vamos a aplicar a los datos de cada grupo en cada columna para obtener un único valor.

In [8]:
dfg = df.groupby(["Categoria"]).aggregate({
    "Altura":np.sum,
    "Cof":np.mean})

dfg

,Altura,Cof
Categoria,,
cap6,5.32,0.530
cap8,1.78,0.970
laboral,3.41,0.585


**Podemos aplicar un gran número de funciones de agregación:**

- [Funciones estadísticas](https://docs.scipy.org/doc/numpy/reference/routines.statistics.html): mean, std, ...

- [Funciones matemáticas](https://docs.scipy.org/doc/numpy/reference/routines.math.html): sum, prod, ...

- Otras funciones: max, min, ...

- [Documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.aggregate.html)


In [9]:
type(dfg) #Alerta! Una agregación genera un dataframe y por lo tanto podemos seguir aplicando lo que ya sabemos

pandas.core.frame.DataFrame

In [10]:
dfg[dfg.Altura>3]

,Altura,Cof
Categoria,,
cap6,5.32,0.530
laboral,3.41,0.585


### Agrupaciones de múltiples columnas

También se pueden realizar agrupaciones de múltiples columnas. Se crean todas las combinaciones de las diversas columnas que existen en el DataFrame. Veamos un ejemplo:

In [37]:
gr = df.groupby(['Sexo',"Categoria"]).mean()
print(gr)
gr.index

                Altura    Cof
Sexo Categoria               
F    cap6        1.770  0.660
M    cap6        1.780  0.270
     cap8        1.780  0.970
     laboral     1.705  0.585


MultiIndex([('F',    'cap6'),
            ('M',    'cap6'),
            ('M',    'cap8'),
            ('M', 'laboral')],
           names=['Sexo', 'Categoria'])

Si queremos realizar un conteo de los elementos, debemos seleccionar

In [39]:
gr = df.groupby(['Sexo',"Categoria"])["Sexo"].count()
print(gr)

Sexo  Categoria
F     cap6         2
M     cap6         1
      cap8         1
      laboral      2
Name: Sexo, dtype: int64


### Multiindice 

A veces, un índice no es suficiente para expresar la meta-información que identifica una o varias columnas. Por ejemplo, una coordenada está compuesta por la latitud y la longitud.

Un 'multiíndice' es una jerarquía de índices.

Agrupar según diferentes criterios resulta en un multiíndice.

In [14]:
gr = df.groupby(['Sexo',"Categoria"]).count()
gr.index

MultiIndex([('F',    'cap6'),
            ('M',    'cap6'),
            ('M',    'cap8'),
            ('M', 'laboral')],
           names=['Sexo', 'Categoria'])

In [15]:
gr.loc["F"] # primer index

,Nombre,Apellidos,Altura,Nacimiento,Cof
Categoria,,,,,
cap6,2,2,2,2,2


In [16]:
gr.loc["cap6"] # dependent index

KeyError: 'cap6'

In [18]:
# Necesitamos invocar un IndexSlice.
gr.loc[pd.IndexSlice[:, 'cap6'],:] 
# https://pandas.pydata.org/docs/reference/api/pandas.IndexSlice.html

,,Nombre,Apellidos,Altura,Nacimiento,Cof
Sexo,Categoria,,,,,
F,cap6,2,2,2,2,2
M,cap6,1,1,1,1,1


En estas situaciones donde queremos acceder a esos datos, la forma más sencilla es simplemente eliminar el indice:

In [20]:
gr = df.groupby(['Sexo',"Categoria"]).count().reset_index()
gr

,Sexo,Categoria,Nombre,Apellidos,Altura,Nacimiento,Cof
0,F,cap6,2,2,2,2,2
1,M,cap6,1,1,1,1,1
2,M,cap8,1,1,1,1,1
3,M,laboral,2,2,2,2,2


### Agregaciones avanzadas

Sobre una agregación, podemos realizar operaciones más allá de las aritméticas. <br/>

Por ejemplo, si deseamos crear un histograma de la distribución de tipos de certificados por ciudad y código postal, necesitamos crear una lista para cada grupo.

In [21]:
df = pd.read_csv("data/data_groups.csv") 
df.head()

,Unnamed: 0,Dni,Nom,CP,Ciutat,Sexe,Tipus certificat cat,Punts
0,0,H61414629,María Dolores Arjona Jove,7800,Eivissa,M,B,73
1,1,S3138381C,Núria Quirós,7511,Ruberts,F,A,40
2,2,J8698188C,Miguel José María Gil Vargas,7340,Alaro,M,A,45
3,3,A48821615,Jordi Chaves Bustamante,7609,Bellavista,F,B,40
4,4,U0247281I,Jana Rosa Collado Menéndez,7006,Palma,M,B,86


In [22]:
dfc = df.groupby(["Ciutat","CP"])["Tipus certificat cat"].apply(list)
dfc

Ciutat      CP  
Alaro       7340    [A, C, B, A, A, B, A, D, A, B, A, B, B, A, B, ...
Ariany      7529    [B, A, B, A, A, B, A, B, A, B, A, A, C, A, B, ...
Bellavista  7609    [B, C, A, A, A, C, A, A, B, A, A, A, B, A, C, ...
Binissalem  7350    [A, B, B, C, C, A, A, A, C, A, C, A, B, B, D, ...
Eivissa     7800    [B, A, A, B, B, B, A, A, A, A, B, C, A, A, B, ...
La Savina   7870    [A, A, A, B, A, A, B, B, A, A, B, A, B, B, A, ...
Mao         7701    [B, C, B, C, B, C, A, B, B, B, A, C, C, A, C, ...
            7703    [B, A, B, C, A, A, B, A, C, B, A, B, C, A, B, ...
Palma       7006    [B, A, B, B, A, A, A, B, B, A, A, B, B, A, B, ...
            7009    [A, D, A, B, C, A, A, C, B, A, A, A, C, A, A, ...
            7013    [A, A, B, C, C, C, C, A, A, A, B, A, C, C, A, ...
Pedruscada  7590    [A, B, A, A, A, A, B, B, C, B, B, B, C, A, A, ...
Ruberts     7511    [A, A, A, B, C, A, B, A, A, B, B, A, C, B, B, ...
Name: Tipus certificat cat, dtype: object

In [23]:
tipusAlaro = dfc.loc[pd.IndexSlice["Alaro",7340]]

values, counts = np.unique(tipusAlaro, return_counts=True)
print(values) # Tipus 
print(counts) # quantitat
print("-"*40)
distribucioAlaro = dict(zip(values,counts)) ## Que fa el ZIP?!
print(distribucioAlaro)

['A' 'B' 'C' 'D']
[57 42 16  5]
----------------------------------------
{'A': 57, 'B': 42, 'C': 16, 'D': 5}


In [24]:
## També podem invocar funcions especifiques! en lloc de una sum, mean, max, etc.
df2 = df.groupby(["Ciutat","CP"]).agg(
        {"Tipus certificat cat": [lambda x: list(x), np.size]}) # lambda !

print(df2)

                                              Tipus certificat cat     
                                                        <lambda_0> size
Ciutat     CP                                                          
Alaro      7340  [A, C, B, A, A, B, A, D, A, B, A, B, B, A, B, ...  120
Ariany     7529  [B, A, B, A, A, B, A, B, A, B, A, A, C, A, B, ...  113
Bellavista 7609  [B, C, A, A, A, C, A, A, B, A, A, A, B, A, C, ...   98
Binissalem 7350  [A, B, B, C, C, A, A, A, C, A, C, A, B, B, D, ...   93
Eivissa    7800  [B, A, A, B, B, B, A, A, A, A, B, C, A, A, B, ...   91
La Savina  7870  [A, A, A, B, A, A, B, B, A, A, B, A, B, B, A, ...   90
Mao        7701  [B, C, B, C, B, C, A, B, B, B, A, C, C, A, C, ...   63
           7703  [B, A, B, C, A, A, B, A, C, B, A, B, C, A, B, ...   45
Palma      7006  [B, A, B, B, A, A, A, B, B, A, A, B, B, A, B, ...   36
           7009  [A, D, A, B, C, A, A, C, B, A, A, A, C, A, A, ...   23
           7013  [A, A, B, C, C, C, C, A, A, A, B, A, C, C, A, .

In [25]:
# Em aquest exemple apliquem una funció que té un criterí més específic:

def miBarem50p(serie):
  up50list = []
  for value in serie.values:
    if value>50:
      up50list.append(value)
  return len(up50list)


df3 = df.groupby(["Ciutat","CP"]).agg(
        {"Punts": [lambda x: miBarem50p(x), np.size]}) # lambda: què és X?

print(df3) #Quina interpretació dels resultats feu?

                     Punts     
                <lambda_0> size
Ciutat     CP                  
Alaro      7340         61  120
Ariany     7529         56  113
Bellavista 7609         43   98
Binissalem 7350         50   93
Eivissa    7800         44   91
La Savina  7870         41   90
Mao        7701         34   63
           7703         18   45
Palma      7006         19   36
           7009         10   23
           7013         19   37
Pedruscada 7590         65  106
Ruberts    7511         36   85


### Ejercicios

**1) Usando el fichero WHO.csv, ¿Cuál es el volumen total de CO2 emitido por cada continente?**

**2) ¿Cuál es el número de paises por continente?**

**3) Carga el fichero climaMallorca.csv: ¿Cual es la temperatura máxima cuando el viento es inferior a 10? ¿Cuántas muestras hay?**

**4) ¿Cual es la temperatura máxima cuando el viento es superior a 10 y inferior a 20? ¿Cuántas muestras hay?**

**5) Del conjunto "who.csv" selecciona al azar: 30 paises y calcula la media de "Net primary school enrolment ratio female (%)" agrupado por Continente**

Nota: la selección de 30 paises se puede hacer con una función del objeto DataFrame

**6) Repite la anterior actividad pero ahora con todos los paises. ¿Sale la misma media?**